# Métricas do câmbio do dólar (2025)

## About

Este projeto tem como objetivo gerar estatísticas básicas para analisar a flutuação de valor do dólar em relação ao real durante o ano de 2025. Para isso, foram utilizadas técnicas de webscraping para obter a série histórica do dólar para o ano. Uma vez obtidos, os valores foram adicionados a uma planilha excel salva em nuvem, que por sua vez está vinculada a um dashboard Google Looker, onde foram geradas as estatísticas. 

O projeto está estruturado de forma a sempre os últimos valores divulgados, portanto, para atualizar o dashboard, basta rodar o código abaixo:


In [1]:
import requests
import pandas as pd
from datetime import datetime
import lxml
import openpyxl

### Definindo as variáveis:

In [2]:
URL = "https://idealsoftwares.com.br/indices/dolar2025.html"
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; GitHub Copilot)"}
wd = "G:\\Meu Drive\\Eric\\Educação\\MBA_USP\\dsc_data"

### Definindo as funções:



In [3]:
def busca_tabela(url):
    resp = requests.get(url, headers=HEADERS, timeout=15)
    resp.raise_for_status()
    resp.encoding = "utf-8"
    tabelas = pd.read_html(resp.text)
    return tabelas

def pick_tabela(tabelas):
    if not tabelas:
        raise ValueError("Não foi possível encontrar tabelas na página.")
    # Trará a primeira tabela do array. Como já foi analisada a página destino, é sabido que há apenas uma tabela, portanto não há a necessidade de ajustes.
    return tabelas[0]

def clean_table(df: pd.DataFrame) -> pd.DataFrame:
    # basic clean: strip string columns and drop fully empty columns/rows
    df = df.dropna(axis=1, how="all").dropna(axis=0, how="all")
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].str.strip()
    return df

In [13]:
if __name__ == "__main__":
    tabelas = busca_tabela(URL)
    df = pick_tabela(tabelas)
    df = clean_table(df)
    out_file = df.to_csv(f"{wd}\\USD2025_stats.csv", index=False)
    print("Planilha salva em:", wd)


Planilha salva em: G:\Meu Drive\Eric\Educação\MBA_USP\dsc_data


C:\Users\Mr_Li\AppData\Local\Temp\ipykernel_72532\2248515830.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabelas = pd.read_html(resp.text)


O processo de webscraping nem sempre reconhece os formatos corretos de cada coluna. Por exemplo, nesta tabela que puxamos, a coluna de data veio como "object". Isso pode gerar problemas ao levar a tabela exportada para outro software. O ideal é corrigirmos o formatos dos arquivos agora, na fonte.

### Pré-processamento dos dados

O Seguido bloco de código analisa os tipos de dado em cada coluna e os formato para os formatos que serão melhor aprtoveitados no nosso BI.

In [14]:
# Verificando os tipos de dados e convertendo a coluna de data. Coluna "Data" veio errado (object)
print(df.dtypes)

dfn["Data"] = pd.to_datetime(df["Data"], format="%d/%m/%Y")
print(dfn.dtypes)

dfn.to_csv(f"{wd}\\USD2025_stats.csv", index=False)
print("Planilha atualizada em:", wd)


Data             object
Dólar Compra    float64
Dólar Venda     float64
dtype: object
Data            datetime64[ns]
Dólar Compra           Float64
Dólar Venda            Float64
dtype: object
Planilha atualizada em: G:\Meu Drive\Eric\Educação\MBA_USP\dsc_data


In [15]:
# Para conferir se o arquivo foi salvo corretamente, podemos imprimir as primeiras linhas da tabela com:

print(dfn.head())

# Algumas estatísticas básicas sobre a coluna "Dólar Venda":
print("Média: ", df["Dólar Venda"].mean())
print("Mediana: ", df["Dólar Venda"].median())
print("Mínimo: ", df["Dólar Venda"].min())
print("Máximo: ", df["Dólar Venda"].max())
print("Moda: ", df["Dólar Venda"].mode())

        Data  Dólar Compra  Dólar Venda
0 2025-10-01        5.3202       5.3208
1 2025-09-30         5.318       5.3186
2 2025-09-29        5.3223       5.3229
3 2025-09-26        5.3439       5.3445
4 2025-09-25        5.3419       5.3425
Média:  5.648441798941799
Mediana:  5.6539
Mínimo:  5.301
Máximo:  6.2086
Moda:  0    5.3208
1    5.3928
2    5.4278
3    5.4571
4    5.5722
5    5.7087
6    5.7788
Name: Dólar Venda, dtype: float64


Neste caso, podemos observar algumas estatísticas básicas extraídas usando a biblioteca Pandas. O valor para moda chama a atenção, pois ele não possui um único valor e sim 7 valores diferentes, elencados em ordem crescente. Isso significa que para este conjunto de entrada existem 7 modas diferentes. Isto não representa um erro necessariamente. No caso do nosso conjunto de dados, podemos ver que há 4 casas decimais, o que gera uma probabilidade muito baixa de que dois valores se repitam.

### Arquivo auxiliar: Tabela calendário

Já temos o nosos objeto de análise. Porém, como a bolsa de valores não abre aos finais de semana, há uma série de buracos no nosso dado, referentes aos finais de semana. Para isso, precisamos de uma planilha com todos os dias do ano e assim poderemos endereçar os zeros na cotação do dólar.

In [23]:
calendario = pd.date_range(start="2025-01-01", end=pd.Timestamp.today(), freq="D")
#print(calendario)
calendario_df = pd.DataFrame(calendario, columns=["Data"])
cdf = calendario_df.merge(dfn, on="Data", how="left")
calendario_df.to_csv(f"{wd}\\USD_Stats_fulldays.csv", index=False)
print("Planilha calendário com cotação do dólar salva em:", wd)
print(cdf.head())

Planilha calendário com cotação do dólar salva em: G:\Meu Drive\Eric\Educação\MBA_USP\dsc_data
        Data  Dólar Compra  Dólar Venda
0 2025-01-01          <NA>         <NA>
1 2025-01-02         6.208       6.2086
2 2025-01-03        6.1557       6.1563
3 2025-01-04          <NA>         <NA>
4 2025-01-05          <NA>         <NA>
